In [ ]:
"""
load dandi://dandi/000026@draft/sub-I58/ses-Hip-CT/micr/sub-I58_sample-01_chunk-01_hipCT.ome.zarr --name "HiP-CT (hemi)"
load dandi://dandi/000026@draft/sub-I58/ses-Hip-CT/micr/sub-I58_sample-01_chunk-02_hipCT.ome.zarr --name "HiP-CT (broca)" --transform dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/xfm/sub-I58_from-XPCT01_to-XPCT02_desc-final_xfm.lta
load dandi://dandi/000026@draft/sub-I58/ses-Hip-CT/micr/sub-I58_sample-01_chunk-03_hipCT.ome.zarr --name "HiP-CT (brainstem)" --transform dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/xfm/sub-I58_from-XPCT01_to-XPCT03_desc-final_xfm.lta
load dandi://linc/000005@draft/rawdata/sub-I58/micr/sub-I58_sample-blockIC2_chunk-401_PC.ome.zarr --name "HiP-CT (internal capsule)" --transform dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/xfm/sub-I58_from-XPCT01_to-IC401_desc-final_xfm.lta
load dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/dwi/sub-I58_sample-hemi_desc-b0_space-XPCT01_dwi.nii.zarr --name "dMRI (hemi)"
load dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/dwi/sub-I58_sample-IC1_desc-b0_space-XPCT01_dwi.nii.zarr --name "dMRI (internal capsule)" --shader blackred
load /Users/balbasty/localdata/linc/lincbrain/000005/derivatives/registration-nitorch/sub-I58/dwi/sub-I58_desc-CSD_space-XPCT01_tractography.trk --name "Tractography (hemi)"
transform dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58/xfm/sub-I58_from-XPCT01_to-RAS_xfm.lta
"""

DANDI26_I58     = "dandi://dandi/000026@draft/sub-I58"
LINC05_I58      = "dandi://linc/000005@draft/rawdata/sub-I58"
LINC05_I58_REG  = "dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58"
LINC05_I58_LOC  = "/Users/balbasty/localdata/linc/lincbrain/000005/derivatives/registration-nitorch/sub-I58"

HIPCT = {
    "hemi":      f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-01_hipCT.ome.zarr",
    "broca":     f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-02_hipCT.ome.zarr",
    "brainstem": f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-03_hipCT.ome.zarr",
    "ic2": {
        "101":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-101_PC.ome.zarr",
        "201":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-201_PC.ome.zarr",
        "401":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-401_PC.ome.zarr",
    }
}

XFM = {
    "HIPCT_01_02":      f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-XPCT02_desc-final_xfm.lta",
    "HIPCT_01_03":      f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-XPCT03_desc-final_xfm.lta",
    "HIPCT_01_401":     f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-IC401_desc-finalfix_xfm.lta",
    "DWI_HIPCT_init":   f"{LINC05_I58_REG}/xfm/sub-I58_from-dwiHemi_to-XPCT01_desc-init_xfm.lta",
    "HIPCT_RAS":        f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-RAS_xfm.lta"
}

DWI = {
    "ic1": {
        "b0":   f"{LINC05_I58_REG}/dwi/sub-I58_sample-IC1_desc-b0_space-XPCT01_dwi.nii.zarr",
    },
    "hemi": {
        "b0":   f"{LINC05_I58_REG}/dwi/sub-I58_sample-hemi_desc-b0_space-XPCT01_dwi.nii.zarr",
        # "trk":  f"{LINC05_I58_REG}/dwi/sub-I58_desc-CSD_space-XPCT01_tractography.trk",
        "trk":  f"{LINC05_I58_LOC}/dwi/sub-I58_desc-CSD_space-XPCT01_tractography.trk",
    }
}

In [ ]:
from ngtools.local.viewer import LocalNeuroglancer

viewer = LocalNeuroglancer()
print('viewer:      ', viewer.get_viewer_url())
print('fileserver:  ', viewer.get_fileserver_url())

viewer.load({"HiP-CT (hemi)":               "zarr://" + HIPCT["hemi"]})
viewer.load({"HiP-CT (broca)":              "zarr://" + HIPCT["broca"]},      transform=XFM["HIPCT_01_02"])
viewer.load({"HiP-CT (brainstem)":          "zarr://" + HIPCT["brainstem"]},  transform=XFM["HIPCT_01_03"])
viewer.load({"HiP-CT (internal capsule)":   "zarr://" + HIPCT["ic2"]["401"]}, transform=XFM["HIPCT_01_401"])
viewer.load({"dMRI (hemi)":                 "zarr://" + DWI["hemi"]["b0"]})
viewer.load({"dMRI (internal capsule)":     "zarr://" + DWI["ic1"]["b0"]},    shader="blackred")
viewer.load({"Tractography (hemi)":         "trk://"  + DWI["hemi"]["trk"]})
viewer.transform(XFM["HIPCT_RAS"], inv=True)

# with viewer.scene() as scene:
#     scene.showSlices = False


viewer:       http://eduroam-int-dhcp-97-118-202.ucl.ac.uk:59407/v/1/
fileserver:   http://10.97.118.202:59408/
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/', 'transform': {'matrix': [[1.0, 0.0, 0.0, -7.565], [0.0, 1.0, 0.0, -7.565], [0.0, 0.0, 1.0, -7.565]], 'inputDimensions': {'z': [np.float64(15.13), 'um'], 'y': [np.float64(15.13), 'um'], 'x': [np.float64(15.13), 'um']}, 'outputDimensions': {'z': [np.float64(15.13), 'um'], 'y': [np.float64(15.13), 'um'], 'x': [np.float64(15.13), 'um']}}}], 'shader': '\n#uicontrol invlerp normalized\nvoid main() {\nemitGrayscale(normalized());\n}\n', 'shaderControls': {'normalized': {'range': [23005.0, 27807.0], 'window': [3412.0, 57812.0]}}, 'name': 'HiP-CT (hemi)'}
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://dandiarchive.s3.amazonaws.com/zarr/a2861b6d-074a-46cc-a221-b6e9a75dc6be/', 'transform': {'matrix': [[1.0, 0.0, 0.0, -3.105], [0.0, 1.0, 

'radio'

In [3]:
import webbrowser

webbrowser.open(viewer.get_viewer_url())
# webbrowser.open(viewer.state(url=True))

True

In [4]:
viewer.space(layer="HiP-CT (internal capsule)")

'radio'

In [5]:
viewer.space(layer="world")

ValueError: ('No layer named:', 'world')